In [8]:
from qiskit import QuantumCircuit
import numpy as np

from qbench.schemes.postselection import benchmark_using_postselection, benchmark_certification_using_postselection
from qbench.schemes.direct_sum import benchmark_using_direct_sum, benchmark_certification_using_direct_sum

In [9]:
### certification experiment for measurement in Hadamard basis using postselection ###

def state_prep():
    circuit = QuantumCircuit(2)
    circuit.h(0)
    circuit.cx(0, 1)
    return circuit.to_instruction()
    

def u_dag():
    circuit = QuantumCircuit(1)
    circuit.h(0)
    return circuit.to_instruction()

def v0_dag():
    circuit = QuantumCircuit(1)
    circuit.x(0)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    
    return circuit.to_instruction()

def v1_dag():
    circuit = QuantumCircuit(1)
    circuit.x(0)
    circuit.ry(2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.x(0)
    return circuit.to_instruction()

def v0_v1_direct_sum_dag():
    circuit = QuantumCircuit(2)
    circuit.p(-np.pi, 0)
    circuit.ry(-2 * np.arcsin(np.sqrt(0.05)), 0)
    circuit.cx(0, 1)
    return circuit.to_instruction()

In [10]:
from qiskit_aer import StatevectorSimulator

simulator = StatevectorSimulator()

In [12]:
postselection_result = benchmark_certification_using_postselection(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_dag=v0_dag(),
    v1_dag=v1_dag(),
    num_shots_per_measurement=100000
)

postselection_result

0.2805806425831202

In [13]:
expected = (1/np.sqrt(2) * np.sqrt(0.95) - 1/np.sqrt(2) * np.sqrt(0.05))**2
expected

0.2820550528229661

In [17]:
direct_sum_result = benchmark_certification_using_direct_sum(
    backend=simulator,
    target=0,
    ancilla=1,
    state_preparation=state_prep(),
    u_dag=u_dag(),
    v0_v1_direct_sum_dag=v0_v1_direct_sum_dag(),
    num_shots_per_measurement=100000
)

direct_sum_result

0.28115

In [24]:
p_succ = expected
print(f"Analytical p_succ = {p_succ}")
print(
f"Postselection: p_succ = {postselection_result}, abs_error ={np.abs(p_succ - postselection_result)}"
)
print(f"Direct sum: p_succ = {direct_sum_result}, abs_error ={np.abs(p_succ - direct_sum_result)}")

Analytical p_succ = 0.2820550528229661
Postselection: p_succ = 0.28153273017562536, abs_error =0.0005223226473407561
Direct sum: p_succ = 0.28286, abs_error =0.0008049471770338879


In [18]:
from qbench.schemes.postselection import (
assemble_certification_postselection_circuits,
compute_probabilities_from_certification_postselection_measurements,
)
circuits = assemble_certification_postselection_circuits(
target=0,
ancilla=1,
state_preparation=state_prep(),
u_dag=u_dag(),
v0_dag=v0_dag(),
v1_dag=v1_dag(),
)

In [19]:
from qiskit_aer.noise import NoiseModel, ReadoutError
error = ReadoutError([[0.75, 0.25], [0.8, 0.2]])
noise_model = NoiseModel()
noise_model.add_readout_error(error, [0])
noise_model.add_readout_error(error, [1])

In [24]:
keys_ordering = ["u_v0", "u_v1"]

all_circuits = [circuits[key] for key in keys_ordering]

counts_noisy = simulator.run(
    all_circuits,
    backend=simulator,
    noise_model=noise_model,
    shots=100000).result().get_counts()

counts_noiseless = simulator.run(
    all_circuits,
    backend=simulator,
    shots=100000).result().get_counts()

In [25]:
prob_succ_noiseless = compute_probabilities_from_certification_postselection_measurements(
    u_v0_counts=counts_noiseless[0],
    u_v1_counts=counts_noiseless[1],)

prob_succ_noisy = compute_probabilities_from_certification_postselection_measurements(
    u_v0_counts=counts_noisy[0],
    u_v1_counts=counts_noisy[1],)

In [26]:
print(prob_succ_noiseless)

0.2819460844157144


In [27]:
print(prob_succ_noisy)

0.7756271756091706
